In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import GridSearchCV

In [2]:
def load_data():
    data = pd.read_csv('rapport_accident_2022.csv', delimiter=',', quotechar='"')
    data = data.drop('NO_SEQ_COLL', axis=1)
    data.columns = data.columns.str.replace('\t', '')
    data.columns = data.columns.str.replace('"', '')
    return data

def preprocess_data(data):
    data = data.drop('CD_ZON_TRAVX_ROUTR', axis=1)
    data = data.drop('AN', axis=1)
    data = data.dropna()
    return data

In [3]:
data = load_data()
data = preprocess_data(data)
print(data.columns)

Index(['MS_ACCDN', 'HR_ACCDN', 'JR_SEMN_ACCDN', 'GRAVITE', 'NB_VICTIMES_TOTAL',
       'NB_VEH_IMPLIQUES_ACCDN', 'REG_ADM', 'VITESSE_AUTOR', 'CD_GENRE_ACCDN',
       'CD_ETAT_SURFC', 'CD_ECLRM', 'CD_ENVRN_ACCDN', 'CD_CATEG_ROUTE',
       'CD_ASPCT_ROUTE', 'CD_LOCLN_ACCDN', 'CD_CONFG_ROUTE', 'CD_COND_METEO',
       'IND_AUTO_CAMION_LEGER', 'IND_VEH_LOURD', 'IND_MOTO_CYCLO', 'IND_VELO',
       'IND_PIETON'],
      dtype='object')


In [4]:
print(data.head())

   MS_ACCDN           HR_ACCDN JR_SEMN_ACCDN  \
0         3  16:00:00-19:59:00           SEM   
1         4  20:00:00-23:59:00           SEM   
2         3  16:00:00-19:59:00           SEM   
5         5  08:00:00-11:59:00           FDS   
6         6  16:00:00-19:59:00           SEM   

                                             GRAVITE  NB_VICTIMES_TOTAL  \
0                       Dommages matériels seulement                  0   
1                       Dommages matériels seulement                  0   
2                       Dommages matériels seulement                  0   
5  Dommages matériels inférieurs au seuil de rapp...                  0   
6  Dommages matériels inférieurs au seuil de rapp...                  0   

   NB_VEH_IMPLIQUES_ACCDN                     REG_ADM VITESSE_AUTOR  \
0                     1.0              Outaouais (07)            90   
1                     1.0            Laurentides (15)            90   
2                     2.0  Abitibi-Témiscamingu

## Préparation des données

In [5]:
# Type de données
data.dtypes

MS_ACCDN                    int64
HR_ACCDN                   object
JR_SEMN_ACCDN              object
GRAVITE                    object
NB_VICTIMES_TOTAL           int64
NB_VEH_IMPLIQUES_ACCDN    float64
REG_ADM                    object
VITESSE_AUTOR              object
CD_GENRE_ACCDN             object
CD_ETAT_SURFC             float64
CD_ECLRM                  float64
CD_ENVRN_ACCDN            float64
CD_CATEG_ROUTE            float64
CD_ASPCT_ROUTE             object
CD_LOCLN_ACCDN            float64
CD_CONFG_ROUTE            float64
CD_COND_METEO             float64
IND_AUTO_CAMION_LEGER      object
IND_VEH_LOURD              object
IND_MOTO_CYCLO             object
IND_VELO                   object
IND_PIETON                 object
dtype: object

### Ici, nous avons des variables catégorielles, binaires et numériques. Nous devrons les traiter différemment lors de la préparation des données.

In [17]:
df = pd.DataFrame(data)
df['NB_VICTIMES_TOTAL'] = df['NB_VICTIMES_TOTAL'].replace(9, 3)
df['VITESSE_AUTOR'] = df['VITESSE_AUTOR'].replace('<50', 40).astype(int)
binary_map = {'O': 1, 'N': 0}
binary_cols = ['IND_AUTO_CAMION_LEGER', 'IND_VELO', 'IND_MOTO_CYCLO', 'IND_VEH_LOURD', 'IND_PIETON']
for col in binary_cols:
    df[col] = df[col].map(binary_map)

# Préparation des encodeurs pour les variables catégorielles
categorical_features = ['HR_ACCDN', 'JR_SEMN_ACCDN', 'GRAVITE', 'REG_ADM', 'CD_GENRE_ACCDN', 'CD_ASPCT_ROUTE', 'VITESSE_AUTOR', 'CD_ETAT_SURFC', 'CD_ECLRM', 'CD_ENVRN_ACCDN', 'CD_CATEG_ROUTE', 'CD_LOCLN_ACCDN', 'CD_COND_METEO']

Pour les variables catégorielles, nous appliquerons un encodage one-hot, qui convertit les chaînes de caractères en variables numériques. Pour les variables binaires, nous les convertissons les valeurs en 0 et 1. Les variables numériques resteront inchangées.

In [18]:
# Séparation des variables explicatives et de la variable étudiée
X = df.drop('NB_VICTIMES_TOTAL', axis=1)
y = df['NB_VICTIMES_TOTAL']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# le ColumnTransformer appliquera l'encodage one-hot aux colonnes catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'  # les autres colonnes numériques restent inchangées
)

In [19]:
# verification si valeurs nulles
print(X_train.isnull().sum())

MS_ACCDN                  0
HR_ACCDN                  0
JR_SEMN_ACCDN             0
GRAVITE                   0
NB_VEH_IMPLIQUES_ACCDN    0
REG_ADM                   0
VITESSE_AUTOR             0
CD_GENRE_ACCDN            0
CD_ETAT_SURFC             0
CD_ECLRM                  0
CD_ENVRN_ACCDN            0
CD_CATEG_ROUTE            0
CD_ASPCT_ROUTE            0
CD_LOCLN_ACCDN            0
CD_CONFG_ROUTE            0
CD_COND_METEO             0
IND_AUTO_CAMION_LEGER     0
IND_VEH_LOURD             0
IND_MOTO_CYCLO            0
IND_VELO                  0
IND_PIETON                0
dtype: int64


In [20]:
# Le pipeline applique d'abord le prétraitement puis l'apprentissage du modèle.
# On utilise un RandomForestRegressor pour prédire le nombre de victimes.

pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestRegressor(random_state=42))
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat', OneHotEncoder(),
                                                  ['HR_ACCDN', 'JR_SEMN_ACCDN',
                                                   'GRAVITE', 'REG_ADM',
                                                   'CD_GENRE_ACCDN',
                                                   'CD_ASPCT_ROUTE',
                                                   'VITESSE_AUTOR',
                                                   'CD_ETAT_SURFC', 'CD_ECLRM',
                                                   'CD_ENVRN_ACCDN',
                                                   'CD_CATEG_ROUTE',
                                                   'CD_LOCLN_ACCDN',
                                                   'CD_COND_METEO'])])),
                ('classifier', RandomForestRegressor(random_state=42))])

In [21]:
# Paramètres à tester pour le Grid search avec validation croisée
param_grid = {
    'classifier__max_depth': [15, 10, 20, None],  # Profondeurs variées plus None pour pas de limite
    'classifier__min_samples_split': [2, 10, 20],  # Min samples pour splitter
    'classifier__min_samples_leaf': [2, 4, 6]    # Min samples par feuille
}

# Grid search avec validation croisée qui teste les différents paramètres pour éviter l'overfitting et améliorer les performances
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

print("Meilleurs paramètres: ", grid_search.best_params_)
print("Meilleur score MSE: ", -grid_search.best_score_)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] END classifier__max_depth=15, classifier__min_samples_leaf=2, classifier__min_samples_split=2; total time=  23.2s
[CV] END classifier__max_depth=15, classifier__min_samples_leaf=2, classifier__min_samples_split=2; total time=  24.1s
[CV] END classifier__max_depth=15, classifier__min_samples_leaf=2, classifier__min_samples_split=2; total time=  23.6s
[CV] END classifier__max_depth=15, classifier__min_samples_leaf=2, classifier__min_samples_split=10; total time=  21.5s
[CV] END classifier__max_depth=15, classifier__min_samples_leaf=2, classifier__min_samples_split=10; total time=  21.1s
[CV] END classifier__max_depth=15, classifier__min_samples_leaf=2, classifier__min_samples_split=10; total time=  21.7s
[CV] END classifier__max_depth=15, classifier__min_samples_leaf=2, classifier__min_samples_split=20; total time=  20.2s
[CV] END classifier__max_depth=15, classifier__min_samples_leaf=2, classifier__min_samples_split=20; 

In [28]:
# Calculer les erreurs sur l'ensemble d'entraînement
y_train_pred = grid_search.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# Calculer les erreurs sur l'ensemble de test
y_test_pred = grid_search.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Train MSE: {train_mse}, Train R2: {train_r2}')
print(f'Test MSE: {test_mse}, Test R2: {test_r2}')

Train MSE: 0.054307198978674635, Train R2: 0.8439197086623476
Test MSE: 0.05894741077297485, Test R2: 0.8275166346308107


Ici nous voyons que le modèle a des performances similaires sur les ensembles d'entraînement et de test, ce qui indique qu'il n'y a pas d'overfitting. Les scores R2 sont proches de 1, ce qui indique que le modèle est capable de prédire les données de test avec une grande précision.

In [29]:
# Prédiction sur l'ensemble de test
y_pred = grid_search.predict(X_test)

# Calcul des métriques
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Error: {mae}')
print(f'R2 Score: {r2}')

Mean Squared Error: 0.05894741077297485
Root Mean Squared Error: 0.24279087868570115
Mean Absolute Error: 0.08034403148488105
R2 Score: 0.8275166346308107


In [27]:
# Calcul de la moyenne des victimes dans l'ensemble d'entraînement
mean_victims = np.mean(y_train)

# Création des prédictions constantes basées sur cette moyenne pour l'ensemble de test
y_pred_baseline = np.full(shape=y_test.shape, fill_value=mean_victims)

# Calcul des métriques de performance pour le modèle de base
mse_baseline = mean_squared_error(y_test, y_pred_baseline)
rmse_baseline = np.sqrt(mse_baseline)
r2_baseline = r2_score(y_test, y_pred_baseline)

# Affichage des résultats
print("Modèle de base prédisant toujours la moyenne des victimes :")
print(f"Valeur moyenne prédite : {mean_victims}")
print(f"Mean Squared Error (MSE) : {mse_baseline}")
print(f"Root Mean Squared Error (RMSE) : {rmse_baseline}")
print(f"R² Score : {r2_baseline}")

Modèle de base prédisant toujours la moyenne des victimes :
Valeur moyenne prédite : 0.28981450062210157
Mean Squared Error (MSE) : 0.34187348915590293
Root Mean Squared Error (RMSE) : 0.584699486194321
R² Score : -0.00034062848357652165


On voit que le modèle de base, qui prédit toujours la moyenne du nombre de victimes, a des performances nettement inférieures à celles du modèle entraîné. Cela confirme que le modèle entraîné est capable de prédire les données de test de manière significativement plus précise.